In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
##upload the original mRNA and methylation
#interated mRNA and methylation (n=1417) without any operation and exclusion of features
methy_sub = pd.read_csv('/Users/mac1/Desktop/methylation_sub.csv')
mRNA_sub = pd.read_csv('/Users/mac1/Desktop/mRNA_sub.csv')
#overall mRNA (n=1980) without any operation and exclusion of features
mRNA_genetic = pd.read_csv('/Users/mac1/Desktop/mRNA_genetic.csv')

In [5]:
##extract ID row index for cohort
keep_ids_overall = mRNA_genetic['PATIENT_ID'] #overall
keep_ids_integrate = methy_sub['PATIENT_ID'] #integrate
##extract feature sets for methylation or mRNA
methy_features = [c for c in methy_sub.columns if c not in ["PATIENT_ID"]]
mRNA_features = [c for c in mRNA_sub.columns if c not in ["PATIENT_ID"]]

In [6]:
##clinical features （overall cohort)
clinical_impute_overall = pd.read_csv("/Users/mac1/Desktop/DATAFEST/clinical_imputed_1.csv")
clinical_impute_overall.shape

(1980, 29)

In [7]:
##clinical feature (integrated cohort)
clinical_impute_integrate = clinical_impute_overall[clinical_impute_overall['PATIENT_ID'].isin(keep_ids_integrate)].copy()
clinical_impute_integrate.shape

(1417, 29)

In [47]:
##gwas feature (overall cohort)
gwas=pd.read_csv('/Users/mac1/Desktop/DATAFEST/genomics_selected_gwas_ensembl.csv',index_col=0)
gwas_overall = gwas[gwas['PATIENT_ID'].isin(keep_ids_overall)].copy()

,PATIENT_ID,ACVRL1_methy,AHNAK_methy,AKT1_methy,ALK_methy,APC_methy,ARID1A_methy,ARID1B_methy,ARID2_methy,ASXL1_methy,...,EMID1_mrna,TTC28_mrna,ATAD5_mrna,PAX9_mrna,MLLT10_mrna,PTHLH_mrna,SKIDA1_mrna,MRPS30_mrna,APOBEC3A_mrna,SEC14L4_mrna
1,MB-0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.825269,5.798647,5.762455,5.469058,7.263043,6.013555,5.608817,8.535214,5.336678,5.868042
2,MB-0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.698300,5.609069,5.807519,6.411290,8.121593,5.764321,6.050104,9.628202,5.377759,5.747465
3,MB-0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.713216,5.485918,5.704256,5.817795,6.877284,9.808890,5.321103,8.915721,5.338215,5.554798
4,MB-0006,0.228235,0.039574,0.012346,0.032258,0.190789,0.000000,0.000000,0.000000,0.0,...,5.501439,5.356186,5.670296,6.377648,7.235799,6.089555,5.639904,9.058717,5.515105,5.881517
5,MB-0008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.018440,5.738390,5.705560,5.596711,7.947678,6.121529,5.328361,9.860345,5.375316,5.278561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,MB-7295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.598353,5.185025,5.718490,6.096267,7.250477,6.841198,5.444054,9.119662,5.289611,5.461226
1982,MB-7296,0.491018,0.065934,NaN,0.000000,0.030303,0.007895,0.007519,0.003571,0.0,...,6.195056,5.512736,5.968755,5.843940,7.143147,5.865243,5.447092,9.195065,5.441914,5.688695
1983,MB-7297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.408007,5.631185,6.096751,5.650109,7.113933,5.667898,5.381733,10.738350,5.311344,5.461644
1984,MB-7298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.629645,5.543150,5.641894,5.715491,7.449389,6.630646,5.391292,12.146450,5.371765,5.449533


In [48]:
#99 methylation in the combined overall gwas feature set, may not be used
gwas_overall_methy = gwas_overall[["PATIENT_ID"] + list(gwas_overall.columns.intersection(methy_features))]
gwas_overall_methy.shape

(1980, 99)

In [49]:
#125 mRNA in the combined overall gwas feature set
gwas_overall_mRNA = gwas_overall[["PATIENT_ID"] + list(gwas_overall.columns.intersection(mRNA_features))]
gwas_overall_mRNA.shape

(1980, 125)

In [50]:
##gwas feature (integrated cohort)
gwas_integrate = gwas[gwas['PATIENT_ID'].isin(keep_ids_integrate)].copy()
gwas_integrate.shape

(1417, 224)

In [51]:
gwas_integrate_methy = gwas_integrate[["PATIENT_ID"] + list(gwas_integrate.columns.intersection(methy_features))]
gwas_integrate_methy.shape

(1417, 99)

In [52]:
gwas_integrate_mRNA = gwas_integrate[["PATIENT_ID"] + list(gwas_integrate.columns.intersection(mRNA_features))]
gwas_integrate_mRNA.shape

(1417, 125)

In [53]:
nan_counts_mRNA = gwas_overall_mRNA.isna().sum()
nan_counts_meth = gwas_integrate_methy.isna().sum()
missing_pct_mRNA= gwas_overall_mRNA.isna().mean() * 100
missing_pct_meth= gwas_integrate_methy.isna().mean() * 100
print("Columns with NaN in mRNA in GWAS overall:", (nan_counts_mRNA > 0).sum())
print("Columns with NaN in methylation in GWAS integrated:", (nan_counts_meth > 0).sum())
print("Percentage of missing in mRNA in GWAS overall",missing_pct_mRNA.describe())
print("Percentage of missing in methylation in GWAS integrated",missing_pct_meth.describe())
##the missingness percentage is acceptable, let's move on for KNN imputation

Columns with NaN in mRNA in GWAS overall: 0
Columns with NaN in methylation in GWAS integrated: 75
Percentage of missing in mRNA in GWAS overall count    125.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
Percentage of missing in methylation in GWAS integrated count    99.000000
mean      3.758830
std       6.782153
min       0.000000
25%       0.070572
50%       0.423430
75%       3.705011
max      28.863797
dtype: float64


In [54]:
##data-driven feature set
#integrated cohort case 1 (n=1417, N=200)
var_data_driven_integrate_1 = pd.read_csv('/Users/mac1/Desktop/DATAFEST/strategy_3_Importance_Cohort1_Case1_SeparateCox.csv')
top200_features_1 = var_data_driven_integrate_1["feature_name"].head(200).tolist()
##integrated cohort case 2 (n=1417, N=300)
var_data_driven_integrate_2 = pd.read_csv('/Users/mac1/Desktop/DATAFEST/strategy_3_Importance_Cohort1_Case2_CombinedCox.csv')
top300_features_2 = var_data_driven_integrate_2["feature_name"].head(300).tolist()
#overall cohort (n=1980, N mRNA=100)
var_data_driven_overall = pd.read_csv('/Users/mac1/Desktop/DATAFEST/strategy_3_Importance_Cohort1_Case2_CombinedCox.csv')
top100_features_3 = var_data_driven_overall["feature_name"].head(100).tolist()

In [55]:
##data-driven dataset (integrate cohort case 1)
genetic_sub = methy_sub.merge(mRNA_sub, on="PATIENT_ID", how="inner")
data_driven_integrate_1 = genetic_sub[["PATIENT_ID"] + list(genetic_sub.columns.intersection(top200_features_1))]
##data-driven dataset (integrate cohort case 2)
data_driven_integrate_2 = genetic_sub[["PATIENT_ID"] + list(genetic_sub.columns.intersection(top300_features_2))]
##data-driven dataset (overall cohort)
data_driven_overall = genetic_sub[["PATIENT_ID"] + list(genetic_sub.columns.intersection(top100_features_3))]

In [56]:
##Combine clinical+gwas+data-driven
#integrate cohort case 1 (N=1417,N=452)
data_all_integrate_1 = clinical_impute_integrate.merge(gwas_integrate, on="PATIENT_ID", how="inner") #clinical+gwas
data_all_integrate_1 = data_all_integrate_1.merge(data_driven_integrate_1, on="PATIENT_ID", how="inner") # +data-driven
#integrate cohort case 2 (n=1417, N=552)
data_all_integrate_2 = clinical_impute_integrate.merge(gwas_integrate, on="PATIENT_ID", how="inner") #clinical+gwas
data_all_integrate_2 = data_all_integrate_2.merge(data_driven_integrate_2, on="PATIENT_ID", how="inner") # +data-driven
#overall cohort (n=1417, N=253)
data_all_overall = clinical_impute_overall.merge(gwas_overall_mRNA, on="PATIENT_ID", how="inner") #clinical+gwas (mrna)
data_all_overall = data_all_overall.merge(data_driven_overall, on="PATIENT_ID", how="inner") # +data-driven

In [58]:
##split into training and testing (70% vs 30%)
#integrate cohort case 1 (N=1417,N=452)
train_all_integrate_1, test_all_integrate_1 = train_test_split(
    data_all_integrate_1,
    test_size=0.30,
    stratify=data_all_integrate_1['OS_STATUS'],
    random_state=123
)
X_train_integrate_1 = train_all_integrate_1.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_1 = train_all_integrate_1[['OS_MONTHS', 'OS_STATUS']]
X_test_integrate_1 = test_all_integrate_1.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_1 = test_all_integrate_1[['OS_MONTHS', 'OS_STATUS']]

#integrate cohort case 2 (N=1417,N=552)
train_all_integrate_2, test_all_integrate_2 = train_test_split(
    data_all_integrate_2,
    test_size=0.30,
    stratify=data_all_integrate_2['OS_STATUS'],
    random_state=123
)
X_train_integrate_2 = train_all_integrate_2.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_2 = train_all_integrate_2[['OS_MONTHS', 'OS_STATUS']]
X_test_integrate_2 = test_all_integrate_2.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_2 = test_all_integrate_2[['OS_MONTHS', 'OS_STATUS']]

#overall cohort case 2 (N=1980,N=253)
train_all_overall, test_all_overall = train_test_split(
    data_all_overall,
    test_size=0.30,
    stratify=data_all_overall['OS_STATUS'],
    random_state=123
)
X_train_overall = train_all_overall.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_overall = train_all_overall[['OS_MONTHS', 'OS_STATUS']]
X_test_overall = test_all_overall.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_overall = test_all_overall[['OS_MONTHS', 'OS_STATUS']]

In [69]:
## KNN imputation (genetic only + std first)
#integrate cohort case 1 (N=1417,N=452)
Xc_train_integrate_1 = X_train_integrate_1.iloc[:, :26]
Xg_train_integrate_1 = X_train_integrate_1.iloc[:, 26:] #only impute on training
Xc_test_integrate_1  = X_test_integrate_1.iloc[:, :26]
Xg_test_integrate_1  = X_test_integrate_1.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_1_scaled = scaler.fit_transform(Xg_train_integrate_1) #std first

imputer = KNNImputer(n_neighbors=5,weights="distance",metric="nan_euclidean") #knn
Xg_train_integrate_1_imp = imputer.fit_transform(Xg_train_integrate_1_scaled)

Xg_train_integrate_1_imp = pd.DataFrame( #get columns' name back
    Xg_train_integrate_1_imp,
    columns=Xg_train_integrate_1.columns,
    index=Xg_train_integrate_1.index)

X_train_integrate_1_imp = pd.concat([Xc_train_integrate_1, Xg_train_integrate_1_imp], axis=1)

Xg_test_integrate_1_scaled = scaler.transform(Xg_test_integrate_1) #Scale and std test using TRAIN statistics
Xg_test_integrate_1_imp = imputer.transform(Xg_test_integrate_1_scaled)
Xg_test_integrate_1_imp = pd.DataFrame(
    Xg_test_integrate_1_imp,
    columns=Xg_test_integrate_1.columns,
    index=Xg_test_integrate_1.index
)
X_test_integrate_1_imp = pd.concat([Xc_test_integrate_1, Xg_test_integrate_1_imp], axis=1)

In [ ]:
## KNN imputation (genetic only + std first)
#integrate cohort case 2 (N=1417,N=552)
Xc_train_integrate_2 = X_train_integrate_2.iloc[:, :26]
Xg_train_integrate_2 = X_train_integrate_2.iloc[:, 26:]  #only impute on training
Xc_test_integrate_2 = X_test_integrate_2.iloc[:, :26]
Xg_test_integrate_2 = X_test_integrate_2.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_2_scaled = scaler.fit_transform(Xg_train_integrate_2)  #std first

imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")  #knn
Xg_train_integrate_2_imp = imputer.fit_transform(Xg_train_integrate_2_scaled)

Xg_train_integrate_2_imp = pd.DataFrame(  #get columns' name back
    Xg_train_integrate_2_imp,
    columns=Xg_train_integrate_2.columns,
    index=Xg_train_integrate_2.index)

X_train_integrate_2_imp = pd.concat([Xc_train_integrate_2, Xg_train_integrate_2_imp], axis=1)

Xg_test_integrate_2_scaled = scaler.transform(Xg_test_integrate_2)  #Scale and std test using TRAIN statistics
Xg_test_integrate_2_imp = imputer.transform(Xg_test_integrate_2_scaled)
Xg_test_integrate_2_imp = pd.DataFrame(
    Xg_test_integrate_2_imp,
    columns=Xg_test_integrate_2.columns,
    index=Xg_test_integrate_2.index
)
X_test_integrate_2_imp = pd.concat([Xc_test_integrate_2, Xg_test_integrate_2_imp], axis=1)

In [74]:
## KNN imputation (genetic only + std first)
#overall cohort (N=1980,N=252)
Xc_train_overall = X_train_overall.iloc[:, :26]
Xg_train_overall = X_train_overall.iloc[:, 26:]  #only impute on training
Xc_test_overall = X_test_overall.iloc[:, :26]
Xg_test_overall = X_test_overall.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_overall_scaled = scaler.fit_transform(Xg_train_overall)  #std first

imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")  #knn
Xg_train_overall_imp = imputer.fit_transform(Xg_train_overall_scaled)

Xg_train_overall_imp = pd.DataFrame(  #get columns' name back
    Xg_train_overall_imp,
    columns=Xg_train_overall.columns,
    index=Xg_train_overall.index)

X_train_overall_imp = pd.concat([Xc_train_overall, Xg_train_overall_imp], axis=1)

Xg_test_overall_scaled = scaler.transform(Xg_test_overall)  #Scale and std test using TRAIN statistics
Xg_test_overall_imp = imputer.transform(Xg_test_overall_scaled)
Xg_test_overall_imp = pd.DataFrame(
    Xg_test_overall_imp,
    columns=Xg_test_overall.columns,
    index=Xg_test_overall.index
)
X_test_overall_imp = pd.concat([Xc_test_overall, Xg_test_overall_imp], axis=1)


In [79]:
X_test_overall_imp

,RFS_MONTHS,RFS_STATUS,VITAL_STATUS,LYMPH_NODES_EXAMINED_POSITIVE,NPI,AGE_AT_DIAGNOSIS,TUMOR_SIZE,TMB_NONSYNONYMOUS,CELLULARITY,CHEMOTHERAPY,...,FXYD4_mrna,TPD52L1_mrna,ENC1_mrna,DNAH11_mrna_y,ALDOC_mrna,GRB2_mrna,GPR156_mrna,VANGL1_mrna,NKX2-2_mrna,UMODL1_mrna
342,121.966667,0:Not Recurred,Living,1,4.042,79.52,21.0,9.152624,High,NO,...,0.992813,1.416946,-1.223732,-1.054836,-1.081067,-0.300710,0.919122,0.161028,-0.428187,0.224989
325,149.866667,0:Not Recurred,Living,0,2.040,61.79,20.0,11.767659,Moderate,NO,...,0.009157,0.942511,0.392749,0.240805,-0.491881,0.271729,-0.483658,0.484974,-0.176472,-0.728284
946,45.533333,1:Recurred,Died of Disease,0,3.032,43.63,16.0,10.460142,Moderate,NO,...,-0.491750,0.745315,0.274270,0.850794,-0.957869,-0.096607,1.658370,-0.290612,-0.415236,-0.821329
1011,187.033333,0:Not Recurred,Died of Other Causes,1,4.030,74.46,15.0,6.537589,High,NO,...,-0.023318,0.906088,-0.305917,-0.715573,-0.390119,-0.838757,0.300666,-1.049251,-0.455549,-0.755907
1168,21.066667,1:Recurred,Died of Disease,3,4.090,60.41,45.0,3.922553,High,NO,...,0.113393,0.353704,0.089972,0.233243,0.291860,0.632116,0.926125,-0.171070,2.668258,-0.582256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,239.166667,0:Not Recurred,Living,6,5.070,58.92,35.0,9.152624,Low,NO,...,0.118282,0.365060,0.822153,0.158290,1.741177,-1.079096,-0.040037,-0.530432,3.234289,-1.201279
965,62.633333,1:Recurred,Died of Disease,1,4.040,69.72,20.0,10.460142,High,NO,...,-0.252079,0.657425,-0.533618,0.059456,-0.672819,0.489529,0.075651,-1.201659,-0.632220,-0.407730
282,130.900000,0:Not Recurred,Living,4,4.052,60.00,26.0,2.615035,High,YES,...,0.101426,1.261980,-0.096220,-0.623950,1.074558,-0.049146,-0.609271,0.196321,-0.558113,1.014398
607,199.966667,0:Not Recurred,Living,0,2.030,49.37,15.0,5.230071,Moderate,NO,...,-0.654629,-0.214460,0.339306,0.374215,0.642533,-0.306414,-0.694937,-0.365147,-0.654322,-0.322543


In [82]:
##output all dataset ready for prediction models
X_train_integrate_1_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_1_imp.csv", index=False)
X_test_integrate_1_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_1_imp.csv", index=False)
y_train_integrate_1.to_csv("/Users/mac1/Desktop/y_train_integrate_1.csv", index=False)
y_test_integrate_1.to_csv("/Users/mac1/Desktop/y_test_integrate_1.csv", index=False)
X_train_integrate_2_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_2_imp.csv", index=False)
X_test_integrate_2_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_2_imp.csv", index=False)
y_train_integrate_2.to_csv("/Users/mac1/Desktop/y_train_integrate_2.csv", index=False)
y_test_integrate_2.to_csv("/Users/mac1/Desktop/y_test_integrate_2.csv", index=False)
X_train_overall_imp.to_csv("/Users/mac1/Desktop/X_train_overall_imp.csv", index=False)
X_test_overall_imp.to_csv("/Users/mac1/Desktop/X_test_overall_imp.csv", index=False)
y_train_overall.to_csv("/Users/mac1/Desktop/y_train_overall.csv", index=False)
y_test_overall.to_csv("/Users/mac1/Desktop/y_test_overall.csv", index=False)

In [81]:
##ID list and feature set
keep_ids_overall.to_frame(name="PATIENT_ID").to_csv(
    "/Users/mac1/Desktop/keep_ids_overall.csv", index=False
)
keep_ids_integrate.to_frame(name="PATIENT_ID").to_csv(
    "/Users/mac1/Desktop/keep_ids_integrate.csv", index=False
)